# Autoregressive Distributed Lag (ARDL) models

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns

sns.set_style("darkgrid")
sns.mpl.rc("figure", figsize=(16, 6))
sns.mpl.rc("font", size=14)

In [ ]:
from statsmodels.datasets.danish_data import load
from statsmodels.tsa.api import ARDL
from statsmodels.tsa.ardl import ardl_select_order

data = load().data
data = data[["lrm", "lry", "ibo", "ide"]]
data.tail()

In [ ]:
_ = (data - data.mean()).plot()

In [ ]:
sel_res = ardl_select_order(
    data.lrm, 3, data[["lry", "ibo", "ide"]], 3, ic="aic", trend="c"
)
sel_res.model.ardl_order

In [ ]:
res = sel_res.model.fit()
res.summary()

In [ ]:
sel_res = ardl_select_order(
    data.lrm, 3, data[["lry", "ibo", "ide"]], 3, ic="bic", trend="c", glob=True
)
sel_res.model.ardl_order

In [ ]:
sel_res.model.ar_lags

In [ ]:
sel_res.model.dl_lags

In [ ]:
for i, val in enumerate(sel_res.bic.head(10)):
    print(f"{i+1}: {val}")

In [ ]:
res = ARDL(
    data.lrm, 2, data[["lry", "ibo", "ide"]], {"lry": 1, "ibo": 2, "ide": 3}, trend="c"
).fit()
res.summary()

In [ ]:
res = ARDL(
    data.lrm,
    2,
    data[["lry", "ibo", "ide"]],
    {"lry": 1, "ibo": 2, "ide": 3},
    trend="c",
    causal=True,
).fit()
res.summary()

In [ ]:
from statsmodels.tsa.api import UECM

sel_res = ardl_select_order(
    data.lrm, 3, data[["lry", "ibo", "ide"]], 3, ic="aic", trend="c"
)

ecm = UECM.from_ardl(sel_res.model)
ecm_res = ecm.fit()
ecm_res.summary()

In [ ]:
ecm_res.ci_summary()

In [ ]:
ecm_res.ci_resids.plot(title="Cointegrating Error")

In [ ]:
ecm = UECM(data.lrm, 2, data[["lry", "ibo", "ide"]], 2, seasonal=True)
seasonal_ecm_res = ecm.fit()
seasonal_ecm_res.summary()

In [ ]:
seasonal_ecm_res.ci_summary()

In [ ]:
seasonal_ecm_res.ci_resids.plot(title="Cointegrating Error with Seasonality")

In [ ]:
greene = pd.read_fwf("http://www.stern.nyu.edu/~wgreene/Text/Edition7/TableF5-2.txt")
greene.head()

In [ ]:
index = pd.to_datetime(
    greene.Year.astype("int").astype("str")
    + "Q"
    + greene.qtr.astype("int").astype("str")
)
greene.index = index
greene.index.freq = greene.index.inferred_freq
greene.head()

In [ ]:
greene["c"] = np.log(greene.realcons)
greene["g"] = np.log(greene.realgdp)

In [ ]:
sel_res = ardl_select_order(
    greene.c, 8, greene[["g"]], 8, trend="c", seasonal=True, ic="aic"
)
ardl = sel_res.model
ardl.ardl_order

In [ ]:
res = ardl.fit(use_t=True)
res.summary()

In [ ]:
uecm = UECM.from_ardl(ardl)
uecm_res = uecm.fit()
uecm_res.summary()

In [ ]:
uecm_res.ci_summary()

In [ ]:
uecm = UECM(greene.c, 3, greene[["g"]], 3, trend="c", seasonal=True)
uecm_res = uecm.fit()
uecm_res.summary()

In [ ]:
uecm_res.ci_summary()

In [ ]:
_ = uecm_res.ci_resids.plot(title="Cointegrating Error")